In [2]:
import cv2
import mediapipe as mp
import threading
import pygame
import imutils
import sys

numw =0
# Initialize hand detection model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Initialize pygame for music playback
pygame.mixer.init()
music_playing = False

# Load the pre-trained HOG descriptor for pedestrian detection
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Shared variables
flag_hand = False
flag_mobile = False

# Open webcam
cap = cv2.VideoCapture(0)

def detect_hands():
    global flag_hand
    global numw
    while True:
        success, image = cap.read()
        if not success:
            break

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            flag_hand = True
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cv2.putText(image, "Hands Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            flag_hand = False
            cv2.putText(image, "Hand Not Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow("Hand Detection", image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            numw=numw+1
            break

def detect_mobile():
    global flag_mobile
    global numw
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = imutils.resize(frame, width=min(400, frame.shape[1]))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Adjust parameters for more accurate detection
        regions, _ = hog.detectMultiScale(gray, winStride=(6, 6), padding=(8, 8), scale=1.05)

        flag_mobile = False
        for (x, y, w, h) in regions:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi = gray[y:y+h, x:x+w]
            _, thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            white_pixels = cv2.countNonZero(thresh)

            if white_pixels > 100:  # Adjust threshold for accuracy
                flag_mobile = True
                cv2.putText(frame, "Talking on mobile phone", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        cv2.imshow("Mobile Phone Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            numw=numw+1
            break

def play_music():
    global music_playing
    global numw
    while True:
        if flag_hand and flag_mobile:
            if not music_playing:
                pygame.mixer.music.load("music.wav")
                pygame.mixer.music.play(-1)
                music_playing = True
        else:
            if numw == 2:
                    break
            if music_playing:
                pygame.mixer.music.stop()
                music_playing = False
                
            
                

# Create threads for each function
hand_thread = threading.Thread(target=detect_hands)
mobile_thread = threading.Thread(target=detect_mobile)
music_thread = threading.Thread(target=play_music)

# Start all threads
hand_thread.start()
mobile_thread.start()
music_thread.start()

# Join all threads to wait for them to finish
hand_thread.join()
mobile_thread.join()
music_thread.join()

# Release resource
cap.release()
cv2.destroyAllWindows()

